In [1]:
import pandas as pd
import re
import io
import os
import glob
import time
from unicodedata import normalize
from sqlalchemy import create_engine
import psycopg2 as ps
from sqlalchemy import text

#database information to connect
host = 'ds4ateam77.cxdzzcrtbiby.us-east-2.rds.amazonaws.com'
port = 5432
user = 'postgres'
password = '12345678'
database = 'teate'
def runQuery(sql):
    engine=create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}',max_overflow=30)
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())


In [2]:
#Formulas to use further
def formatcolumn(text,flag = True):
    text = text.lower()
    text = re.sub('<.*?>', '', text)
    text = re.sub(':.*?:', '', text)
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    text = normalize( 'NFC', text)
    if flag:
        text = re.sub('[^a-z0-9 ]', '', text)
    else:
        text = re.sub('[^a-z0-9]', '', text)
    return text

In [3]:
#Read Data and format for upload
data = pd.read_csv('upload/back_up_categ.csv',encoding = 'unicode_escape',dtype='str') 
data.columns = [formatcolumn(val,False) for val in data.columns]
_N='\n'
_R='\r'
_E=''
data= data.replace(_N,_E, regex=True)
data= data.replace(_R,_E, regex=True)
data=data.reset_index()

In [4]:
#connect drop table if exists and create it 
TABLE_NAME = 'categories_backup'
connDB = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = connDB.raw_connection()
cursor = conn.cursor()
data.head(0).to_sql(TABLE_NAME, connDB, if_exists='replace',index=False)
conn.commit()
conn.close()

#Upload Data in cycles
S_=0
D_=1000
start_time = time.time()
print("Start at 0 seconds ---")
for n in range(S_,len(data),D_):
    S_=n
    E_=n+D_
    df2=data[(data.index>=S_) & (data.index<E_)]
    #connect to db
    conn = connDB.raw_connection()
    cursor = conn.cursor()
    engine=connDB
    conn = engine.raw_connection()
    cur = conn.cursor()
    output = io.StringIO()
    df2.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    cur.copy_from(output, TABLE_NAME, null="" , sep='\t') # null values become ''
    conn.commit()
    Comp = round((n+D_)/len(data)*100,1)
    Records = str(n+D_)
    print(Records+" "+str(min(Comp,100))+" pct--- %s seconds" % round((time.time() - start_time),4))
df2=data[(data.index>=E_) & (data.index<=len(data))]
print('Done 100%')
print("--- %s seconds ---" % (time.time() - start_time))
conn.close()


Start at 0 seconds ---
1000 38.6 pct--- 0.8685 seconds
2000 77.2 pct--- 1.4723 seconds
3000 100 pct--- 2.2019 seconds
Done 100%
--- 2.205409288406372 seconds ---


In [6]:
start_time = time.time()
SQL_Query = runQuery("""

select * from categories

""") 
print(str(len(SQL_Query))+' rows')
print("--- %s seconds ---" % (time.time() - start_time))
SQL_Query


2606 rows
--- 4.458688735961914 seconds ---


,index,material,jerarquaproductos,nombrecat,nuevacategoria,subcategoria,nombresub,nuevasubcategoria,textobrevedematerial,textobrevedematerial1
0,0,7,00005,CONFITERIA,CONFITERIA Y SNACKS,500005,PASABOCAS,PASABOCAS,Paqte Mani Con Sal x 50 g,PAQTE MANI CON SAL X 50 G
1,1,8,00005,CONFITERIA,CONFITERIA Y SNACKS,500005,PASABOCAS,PASABOCAS,Mani Con Pasas x 50 g,MANI CON PASAS X 50 G
2,2,9,00005,CONFITERIA,CONFITERIA Y SNACKS,500005,PASABOCAS,PASABOCAS,Paqte Mani Confitado x 40 g,PAQTE MANI CONFITADO X 40 G
3,3,10,00005,CONFITERIA,CONFITERIA Y SNACKS,500005,PASABOCAS,PASABOCAS,Paqte Mani Japones x 40 g,PAQTE MANI JAPONES X 40 G
4,4,11,00005,CONFITERIA,CONFITERIA Y SNACKS,500005,PASABOCAS,PASABOCAS,Paqte Mix Original x 50 g,PAQTE MIX ORIGINAL X 50 G
5,5,12,00003,ALIMENTOS,ALIMENTOS,300022,MEZCLA PARA TORTAS,HARINAS Y MEZCLAS PARA PREPARAR,Harina Haz de Oro x 200g,HARINA HAZ DE ORO X 200G
6,6,13,00003,ALIMENTOS,ALIMENTOS,300022,MEZCLA PARA TORTAS,HARINAS Y MEZCLAS PARA PREPARAR,Harina Haz de Oro x 500g,HARINA HAZ DE ORO X 500G
7,7,14,00003,ALIMENTOS,ALIMENTOS,300022,MEZCLA PARA TORTAS,HARINAS Y MEZCLAS PARA PREPARAR,Harina Haz de Oro x 500 g polvo de horne,HARINA HAZ DE ORO X 500 G POLVO DE HORNE
8,8,15,00003,ALIMENTOS,ALIMENTOS,300026,PASTAS,"ARROZ, GRANOS Y PASTA",Pasta Mueca Espaguetti x 250g,PASTA MUECA ESPAGUETTI X 250G
9,9,16,00003,ALIMENTOS,ALIMENTOS,300026,PASTAS,"ARROZ, GRANOS Y PASTA",Pasta Mueca Concha x 250 g,PASTA MUECA CONCHA X 250 G
